In [5]:
from importlib import reload
import utils
import models
from time import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
# load the corpus
corpus = utils.AMI_Corpus(seed = 75, embed_vec = 'glove300')

Loading corpus from /home/jake_miller/final/nlp-final-project/data
Begin corpus post-processing ...
Splitting corpus into training and test ...
Creating vocabulary from training set ...
Found 8435 unique words.
Building initial embedding matrix ...
(8437, 300)
loading pretrained vectors from glove.6B.300d.txt


# First Up: LSTM-Softmax

In [10]:
def run_lstm_model():
    
    from tensorflow.keras.callbacks import EarlyStopping

    BATCH_SIZE = 512
    EPOCHS = 25

    # choose hyperparameters
    num_layers = 3 
    hidden_state_size = 150
    dropout_rate = 0.4
    bidirectional = True
    stateful = False
    trainable_embed = True

    lstm = models.LSTMSoftmax(corpus,
                              batch_size = BATCH_SIZE,
                              num_layers = num_layers,
                              dropout_rate = dropout_rate,
                              hidden_state_size = hidden_state_size,
                              stateful = stateful, bidirectional = bidirectional, trainable_embed = trainable_embed)

    lstm.model.compile(optimizer = 'adagrad', metrics = ['acc'], loss = 'categorical_crossentropy')

    # create our generators
    ug_train = utils.UtteranceGenerator(corpus, "train", batch_size = BATCH_SIZE, algo = "LSTM_Soft")
    ug_val = utils.UtteranceGenerator(corpus, "val", batch_size = BATCH_SIZE, algo = "LSTM_Soft")
    ug_test = utils.UtteranceGenerator(corpus, "test", batch_size = BATCH_SIZE, algo = "LSTM_Soft")

    # create keras callbacks
    es = EarlyStopping(monitor='val_loss', patience=5, verbose=0, restore_best_weights = True)

    train_start = time()
    
    # note to self, maybe change validation_steps and validation_freq
    history = lstm.model.fit_generator(ug_train, epochs=EPOCHS, verbose=1, callbacks = [es], validation_data=ug_val, 
                                       use_multiprocessing=False, shuffle=True)
    train_end = time()
    
    results = lstm.model.evaluate_generator(ug_test)
    
    test_end = time()
    

    return [(train_end - train_start), (test_end - train_end)] + results


In [12]:
lstm_results = run_lstm_model()

Epoch 1/25
156/156 [==============================] - 11s 70ms/step - loss: 1.7047 - acc: 0.4397 - val_loss: 1.4419 - val_acc: 0.5511
Epoch 2/25
156/156 [==============================] - 10s 62ms/step - loss: 1.3545 - acc: 0.5601 - val_loss: 1.3003 - val_acc: 0.5742
Epoch 3/25
156/156 [==============================] - 10s 61ms/step - loss: 1.2519 - acc: 0.5831 - val_loss: 1.2255 - val_acc: 0.5823
Epoch 4/25
156/156 [==============================] - 10s 61ms/step - loss: 1.1912 - acc: 0.6046 - val_loss: 1.1798 - val_acc: 0.6107
Epoch 5/25
156/156 [==============================] - 10s 61ms/step - loss: 1.1428 - acc: 0.6201 - val_loss: 1.1334 - val_acc: 0.6276
Epoch 6/25
156/156 [==============================] - 10s 61ms/step - loss: 1.1036 - acc: 0.6305 - val_loss: 1.1072 - val_acc: 0.6288
Epoch 7/25
156/156 [==============================] - 10s 61ms/step - loss: 1.0690 - acc: 0.6406 - val_loss: 1.0811 - val_acc: 0.6376
Epoch 8/25
156/156 [==============================] - 10s 61ms

# Next, CNN model

In [16]:
def run_cnn_model():
    
    from tensorflow.keras.callbacks import EarlyStopping

    BATCH_SIZE = 512
    EPOCHS = 25

    # choose hyperparameters
    d1 = 0
    d2 = 2
    filters = 200
    kernel_size = 3
    hidden_units = 140
    dropout_rate = 0.5
    trainable_embed = False
    embed_vec = 'glove300'

    cnn = models.CNN(corpus = corpus, d1 = d1, d2 = d2,
                       batch_size = BATCH_SIZE,
                       filters = filters,
                       kernel_size = kernel_size,
                       hidden_units = hidden_units,
                       dropout_rate = dropout_rate,
                       trainable_embed = trainable_embed)

    cnn.model.compile(optimizer = 'adagrad', metrics = ['acc'], loss = 'categorical_crossentropy')

    # create our generators
    ug_train = utils.UtteranceGenerator(corpus, "train", batch_size = BATCH_SIZE, algo = "CNN", sequence_length = 3)
    ug_val = utils.UtteranceGenerator(corpus, "val", batch_size = BATCH_SIZE, algo = "CNN", sequence_length = 3)
    ug_test = utils.UtteranceGenerator(corpus, "test", batch_size = BATCH_SIZE, algo = "CNN", sequence_length = 3)

    # create keras callbacks
    es = EarlyStopping(monitor='val_loss', patience=5, verbose=0, restore_best_weights = True)

    train_start = time()
    
    # note to self, maybe change validation_steps and validation_freq
    history = cnn.model.fit_generator(ug_train, epochs=EPOCHS, verbose=1, callbacks = [es], validation_data=ug_val, 
                                       use_multiprocessing=False, shuffle=True)
    train_end = time()
    
    results = cnn.model.evaluate_generator(ug_test)
    
    test_end = time()
    
    return [(train_end - train_start), (test_end - train_end)] + results

In [17]:
cnn_results = run_cnn_model()

Epoch 1/25
156/156 [==============================] - 4s 29ms/step - loss: 3.5508 - acc: 0.4418 - val_loss: 1.3552 - val_acc: 0.5660
Epoch 2/25
156/156 [==============================] - 3s 19ms/step - loss: 1.3558 - acc: 0.5507 - val_loss: 1.3040 - val_acc: 0.5644
Epoch 3/25
156/156 [==============================] - 3s 19ms/step - loss: 1.2837 - acc: 0.5698 - val_loss: 1.2298 - val_acc: 0.5891
Epoch 4/25
156/156 [==============================] - 3s 19ms/step - loss: 1.2351 - acc: 0.5839 - val_loss: 1.2042 - val_acc: 0.5990
Epoch 5/25
156/156 [==============================] - 3s 19ms/step - loss: 1.1943 - acc: 0.5983 - val_loss: 1.1695 - val_acc: 0.6125
Epoch 6/25
156/156 [==============================] - 3s 19ms/step - loss: 1.1684 - acc: 0.6083 - val_loss: 1.1388 - val_acc: 0.6171
Epoch 7/25
156/156 [==============================] - 3s 19ms/step - loss: 1.1402 - acc: 0.6142 - val_loss: 1.1244 - val_acc: 0.6243
Epoch 8/25
156/156 [==============================] - 3s 19ms/step - 

# Model 3, the LSTM-CRF

In [31]:
def run_lstmcrf_model():
    
    from models_crf import BiLSTMCRF, UtteranceGenerator
    from keras.callbacks import EarlyStopping

    BATCH_SIZE = 512
    EPOCHS = 25

    # choose hyperparameters
    num_layers = 1
    hidden_state_size = 300
    dropout_rate = 0.4
    bidirectional = True
    trainable_embed = True
    embed_vec = 'glove300'
    sequence_length = 7

    lstm_crf = BiLSTMCRF(corpus, batch_size = BATCH_SIZE,
                         sequence_length = sequence_length,
                         num_layers = num_layers,
                         dropout_rate = dropout_rate,
                         hidden_state_size = hidden_state_size,
                         bidirectional = bidirectional, 
                         trainable_embed = trainable_embed)

    lstm_crf.compile()

    # create our generators
    ug_train = UtteranceGenerator(corpus, "train", batch_size = BATCH_SIZE, algo = "LSTM_CRF", sequence_length = sequence_length)
    ug_val = UtteranceGenerator(corpus, "val", batch_size = BATCH_SIZE, algo = "LSTM_CRF", sequence_length = sequence_length)
    ug_test = UtteranceGenerator(corpus, "test", batch_size = BATCH_SIZE, algo = "LSTM_CRF", sequence_length = sequence_length)

    # create keras callbacks
    es = EarlyStopping(monitor='val_loss', patience=5, verbose=0, restore_best_weights = True)

    train_start = time()
    
    # note to self, maybe change validation_steps and validation_freq
    history = lstm_crf.model.fit_generator(ug_train, epochs=EPOCHS, verbose=1, callbacks = [es], validation_data=ug_val, 
                                       use_multiprocessing=False, shuffle=True)
    train_end = time()
    
    results = lstm_crf.model.evaluate_generator(ug_test)
    
    test_end = time()
    
    return [(train_end - train_start), (test_end - train_end)] + results

In [32]:
lstm_crf = run_lstmcrf_model()

Epoch 1/25
155/155 [==============================] - 25s 158ms/step - loss: 1.8756 - crf_viterbi_accuracy: 0.3617 - val_loss: 1.4914 - val_crf_viterbi_accuracy: 0.5116
Epoch 2/25
155/155 [==============================] - 15s 94ms/step - loss: 1.4077 - crf_viterbi_accuracy: 0.5275 - val_loss: 1.3243 - val_crf_viterbi_accuracy: 0.5549
Epoch 3/25
155/155 [==============================] - 14s 93ms/step - loss: 1.2898 - crf_viterbi_accuracy: 0.5643 - val_loss: 1.2491 - val_crf_viterbi_accuracy: 0.5761
Epoch 4/25
155/155 [==============================] - 14s 93ms/step - loss: 1.2158 - crf_viterbi_accuracy: 0.5788 - val_loss: 1.2814 - val_crf_viterbi_accuracy: 0.5192
Epoch 5/25
155/155 [==============================] - 15s 94ms/step - loss: 1.1625 - crf_viterbi_accuracy: 0.5887 - val_loss: 1.1560 - val_crf_viterbi_accuracy: 0.5910
Epoch 6/25
155/155 [==============================] - 14s 93ms/step - loss: 1.1109 - crf_viterbi_accuracy: 0.6003 - val_loss: 1.1297 - val_crf_viterbi_accuracy

# Naive Bayes, just because

In [47]:
reload(models)
def mnb():
    mnb = models.NaiveBayes(corpus, ngram_range = (1,2), tfidf = True)
    
    train_start = time()
    
    mnb.train()
    
    train_end = time()
    
    acc = mnb.eval_on_test()
    
    test_end = time()
    
    return [(train_end - train_start), (test_end - train_end), np.nan, acc]

In [48]:
nb_results = mnb()

# Comparison

In [50]:
import numpy as np
import pandas as pd
pd.DataFrame({"LSTM-Softmax":lstm_results, "CNN":cnn_results, "LSTM-CRF":lstm_crf,
              "Naive Bayes":nb_results}).T

,0,1,2,3
LSTM-Softmax,223.217211,1.126713,1.031624,0.645404
CNN,70.461788,0.338728,1.081208,0.625740
LSTM-CRF,319.369356,1.696913,1.009526,0.623276
Naive Bayes,0.065760,0.015556,NaN,0.475121
